In [5]:
import sys
sys.path.append("../src")

import pandas as pd
import numpy as np
import spacy
from models.SIM import SIM
import utils
import pickle

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def loadObject(file_path):
    with open(file_path,'rb') as f:
        return pickle.load(f)

In [7]:
loadObject(r"..\Data\train_final.pkl")

,ans,equations,original_text,is_valid
43,15,unkn: n\r\nequ: 5*n - 60 = n,I think of a number. I multiply the number by ...,True
104,23Â°49â€™,unkn: x\r\nequ: x=71Â°18'-47Â°29',"How do you add/subtract degrees, minutes, and ...",True
105,746.75,unkn: x\r\nequ: x= 500+1.645*âˆš22500,Can someone please help me with this statistic...,True
106,0.75; 0.389,"unkn: x, y\r\nequ: x = 27/(6*6)\r\nequ: y = 14...",U are tossing 2 fair number cubes number 1 to ...,True
254,{15; 6},"unkn: c, b\r\nequ: c = 9 + b\r\nequ: c-3 = 4(b-3)",How do you write out an equation for a word pr...,True
255,260; 270,"unkn: x,y\r\nequ: 45s + 35b = 51750.00\r\nequ:...",Math Word Problem?\nguys how would i work some...,True
256,4.8,unkn: t\r\nequ: t = 0.200m\r\nequ: m + t = 28.80,"word problem?\nAt chez rob,Geneva was so impre...",True
257,3,unkn: t2\r\nequ: t1 = 6\r\nequ: s1*t1 = (s1+s2...,word problem?\nBetty and Karen have been hired...,True
258,3.316E2,unkn: k\r\nequ: k = 15/3.7854*52*1.6093,Math word Problem with miles to km and galon t...,True
259,{6.2; 1.6},"unkn: x,y\r\nequ: x = 2y + 3\r\nequ: xy = 10\r...",Math Word Problem....?\nThe length of a rectan...,True


In [6]:
math_train = pd.read_json(r"..\Data\dolphin-number_word_std\number_word_std.dev.json")
math_test = pd.read_json(r"..\Data\dolphin-number_word_std\number_word_std.test.json")

In [7]:
model = SIM()
model.fit(math_train)

SIM()

In [8]:
model.score(math_train)

0.17379679144385027

In [4]:
# brus evaluation
res = model.predict(math_train)
correct,total = 0,0
for _,row in res.iterrows():
    #if row['correct']:
    total += 1
    same=True
    for pred,real in zip(row['equations'],row['predicted_equations']):
        if pred.strip()!=real.strip():
            same=False
#             print(row.name)
#             print(row['equations'])
#             print(row['predicted_equations'])
#             print()
            break
    if same:
        correct += 1
correct,total,correct/total

(79, 374, 0.21122994652406418)

In [ ]:
# noam evaluation
corpus_df = math_train

def solve(problem):
    try:
        return utils.solve_eq_string(problem["equations"], integer_flag=utils.is_number(problem["text"]))
    except Exception as e:
        print(problem["equations"])
        print(e)
        print()
        return []

reals_ans = corpus_df['ans_simple']
preds_ans = corpus_df.apply(solve,axis=1)

correct, total = 0, 0
for real_ans,pred_ans in zip(reals_ans,preds_ans):
    if utils.is_same_result(real_ans,pred_ans):
        correct += 1
    total += 1

correct,total,correct/total